# Control-Z Gate Sequence. Emulation using Pulser

## Introduction

In this tutorial we show how to prepare the pulse sequence that generates de *Controlled - Z* gate. We will prepare our state with atoms in any of the "digital" states that we shall call $|g\rangle$ and $|h \rangle$ ( for "ground" and "hyperfine", respectively). Then we will use the *Rydberg blockade* effect to create the logic gate. The levels that each atom can take are the following: 
<img src="files/three_states.png" alt="Three-state Configuration" style="width: 120px;"/>

We will be using *NumPy* and *Matplotlib* for calculations and plots. Additionally, we shall use the library *QuTiP* for creating several quantum objects (and also implicitly while using the simulation option in Pulser)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qutip
from itertools import product

We import the following Classes from Pulser:

In [ ]:
from pulser import Pulse, Sequence, Register
from pulser.devices import Chadoq2
from pulser.simulation import Simulation
from pulser.waveforms import BlackmanWaveform,ConstantWaveform

## 1. Loading the Register on a Pasqal Device

Defining an atom register can simply done by choosing one of the predetermined shapes included in the `Register`class. We can also construct a dictionary with specific labels for each atom. We place our atoms at a distance of $6 \mu m$ which is inside the Rydberg blockade volume

In [ ]:
# Atom Register and Device
q_dict = {"control":np.array([-3.,0.]),
          "target": np.array([3.,0.]),
          }
reg = Register(q_dict)
reg.draw()

## 2. State Preparation

The first part of our sequence will correspond to preparing the different states on which the CZ gate will act. For this, we define the following `Pulse` instances that correspond to $\pi$ and $2\pi$ pulses (notice that the area can be easily fixed using the predefined `BlackmanWaveform`:

Let us construct a function that takes the label string (or "id") of a state and turns it into a ket state. This ket can be in any of the "digital" (ground-hyperfine levels), "ground-rydberg" or "all" levels.

In [ ]:
def build_state_from_id(s_id, basis_name):
    if basis_name not in {'ground-rydberg','digital','all'}:
        raise ValueError('Not a valid basis')
        
    if basis_name == 'digital':
        pool = {''.join(x) for x in product('hg', repeat=2)}  # Gives all string labels
        if s_id not in pool:
            raise ValueError('Not a valid state id for the given basis.')
        ket = {'g' : qutip.basis(2,0),
               'h' : qutip.basis(2,1)}
    
    elif basis_name == 'ground-rydberg':
        pool = {''.join(x) for x in product('gr', repeat=2)}
        if s_id not in pool:
            raise ValueError('Not a valid state id for the given basis.')
        ket = {'r' : qutip.basis(2,0),
               'g' : qutip.basis(2,1)}
        
    elif basis_name == 'all':
        pool = {''.join(x) for x in product('hgr', repeat=2)}
        if s_id not in pool:
            raise ValueError('Not a valid state id for the given basis.')
        ket = {'r' : qutip.basis(3,0),
               'g' : qutip.basis(3,1),
               'h' : qutip.basis(3,2)}
        
    return qutip.tensor([ket[x] for x in s_id])

We try this out:

In [ ]:
build_state_from_id('hg','digital')

Let's now write the state preparation sequence:

In [ ]:
duration = 2e3 # 1µs
pi_Y = Pulse(BlackmanWaveform(duration,np.pi),ConstantWaveform(duration,0),-np.pi/2)
pi_Y.draw()

In [ ]:
def prep_sequence(state_id, seq, duration=200):
    if not isinstance(seq, Sequence):
        raise TypeError('Not a valid Pulser sequence.')
    
    if state_id == 'gg': 
        basis = 'ground-rydberg'
    else:
        basis = 'all'
        
    prep_state = build_state_from_id(state_id, basis)  # Will raise error if not a valid `state_id` 
    
    if state_id != 'gg':
        #    seq.delay(duration, 'raman')
        if state_id[1] == 'h': # Target will be last character in `state_id` string
            seq.declare_channel('raman','raman_local','target')
            seq.add(pi_Y,'raman') 
        if state_id[0] == 'h': 
            if 'raman' not in seq.declared_channels:
                seq.declare_channel('raman','raman_local','control')
            else:
                seq.target('control','raman')
            seq.add(pi_Y,'raman') 
    else:
        print('Warning: `gg` state does not require a preparation sequence.')
        
    return prep_state

Let's test this sequence. Notice that the state "gg" (both atoms in the ground state) is automatically fed to the Register so a pulse sequence is not needed to prepare it.

In [ ]:
# Define sequence and Set channels
seq = Sequence(reg, Chadoq2)
prep_state = prep_sequence('hh',seq)
seq.draw()
print(prep_state)

## 3. Constructing the Gate Sequence

We apply the common $\pi-2\pi-\pi$ sequence for the CZ gate 

In [ ]:
pi_pulse = Pulse(BlackmanWaveform(duration,np.pi),ConstantWaveform(duration,0),0)
twopi_pulse = Pulse(BlackmanWaveform(duration,2*np.pi),ConstantWaveform(duration,0),0)

In [ ]:
def CZ_sequence(initial_id):
    # Define sequence
    seq = Sequence(reg, Chadoq2)
    
    # Prepare State
    prep_state = prep_sequence(initial_id, seq)
    prep_time = max((seq._last(ch).tf for ch in seq.declared_channels), default=0)
    
    # Declare Rydberg channel
    seq.declare_channel('ryd_control', 'rydberg_local', 'control')
    seq.declare_channel('ryd_target', 'rydberg_local2', 'target')
    
    # Write CZ sequence:
    seq.add(pi_pulse, 'ryd_control', 'wait-for-all')  # Wait for state preparation to finish.
    seq.align('ryd_control', 'ryd_target')  # Sets starting time of target channel equal to control channel
    seq.add(twopi_pulse, 'ryd_target')
    seq.align('ryd_control', 'ryd_target')
    seq.add(pi_pulse, 'ryd_control') 

    return seq, prep_state, prep_time

In [ ]:
seq, prep_state, prep_time = CZ_sequence('hh')
#seq.draw()
print(f'Prepared state: {prep_state}')
print(f'Preparation time: {prep_time}ns')

## 4. Simulating the CZ sequence

In [ ]:
for state_id in {'gg','hg','gh','hh'}:
    # Get CZ sequence
    print(f"\n CZ gate sequence acting on state {state_id}\n")
    seq, prep_state, prep_time = CZ_sequence(state_id)
    
    # Construct Simulation instance
    simul = Simulation(seq)
    
    simul.run(progress_bar = True)
    
    data=[st.overlap(prep_state) for st in simul.output.states]
    
    #plt.plot(np.imag(data))
    plt.figure()
    plt.plot(np.real(data))
    plt.xlabel(r"Time [ns]")
    plt.ylabel(fr'$ {state_id} | \psi(t)\rangle$')
    plt.axvspan(0, prep_time, alpha=0.06, color='royalblue')
    plt.title(fr"Action on state $|${state_id}$\rangle$")

Let's do a short version to gather the results:

In [ ]:
CZ = {}
for state_id in {''.join(x) for x in product('gh', repeat=2)}:
    seq, prep_st, prep_t = CZ_sequence(state_id)
    simul = Simulation(seq)
    simul.run()
    final_st = simul.output.states[-1]
    CZ[state_id] = final_st.overlap(prep_st)

In [ ]:
CZ

##  5. CCZ Gate

The same principle can be applied for composite gates. As an application, let us construct the *CCZ* gate, which determines the phase depending on the level of *two* control atoms.

In [ ]:
# Atom Register and Device
q_dict_CCZ = {"control1":np.array([-6.,0.]),
              "target": np.array([0.,2.]),
              "control2": np.array([6.,0.])}
reg_CCZ = Register(q_dict_CCZ)
reg_CCZ.draw()

In [ ]:
def build_state_from_id_CCZ(s_id, basis_name):
    """ Recall that s_id = 'C1'+'C2'+'T' while in the register reg_id = 'C1'+'T'+'C2'."""
    if basis_name not in {'ground-rydberg','digital','all'}:
        raise ValueError('Not a valid basis')
        
    if basis_name == 'digital':
        pool = {''.join(x) for x in product('hg', repeat=3)}  # Gives all string labels
        if s_id not in pool:
            raise ValueError('Not a valid state id for the given basis.')
        ket = {'g' : qutip.basis(2,0),
               'h' : qutip.basis(2,1)}
    
    elif basis_name == 'ground-rydberg':
        pool = {''.join(x) for x in product('gr', repeat=3)}
        if s_id not in pool:
            raise ValueError('Not a valid state id for the given basis.')
        ket = {'r' : qutip.basis(2,0),
               'g' : qutip.basis(2,1)}
        
    elif basis_name == 'all':
        pool = {''.join(x) for x in product('hgr', repeat=3)}
        if s_id not in pool:
            raise ValueError('Not a valid state id for the given basis.')
        ket = {'r' : qutip.basis(3,0),
               'g' : qutip.basis(3,1),
               'h' : qutip.basis(3,2)}
        
    reg_id = s_id[0]+s_id[2]+s_id[1] 
    return qutip.tensor([ket[x] for x in reg_id])

In [ ]:
def prep_sequence_CCZ(state_id, seq, duration=200):
    if not isinstance(seq, Sequence):
        raise TypeError('Not a valid Pulser sequence.')
    
    if state_id == 'ggg': 
        basis = 'ground-rydberg'
    else:
        basis = 'all'
        
    prep_state = build_state_from_id_CCZ(state_id, basis)  # Will raise error if not a valid `state_id` 
    
    if state_id != 'ggg':
        #    seq.delay(duration, 'raman')
        if state_id[2] == 'h': # Target will be last character in `state_id` string
            seq.declare_channel('raman','raman_local','target')
            seq.add(pi_Y,'raman') 
        if state_id[1] == 'h': 
            if 'raman' not in seq.declared_channels:
                seq.declare_channel('raman','raman_local','control2')
            else:
                seq.target('control2','raman')
            seq.add(pi_Y,'raman')
        if state_id[0] == 'h':
            if 'raman' not in seq.declared_channels:
                seq.declare_channel('raman','raman_local','control1')
            else:
                seq.target('control1','raman')
            seq.add(pi_Y,'raman')
    else:
        print('Warning: `ggg` state does not require a preparation sequence.')
        
    return prep_state

In [ ]:
seq = Sequence(reg_CCZ, Chadoq2)
prep_sequence_CCZ('hhg',seq)
seq.draw()

In [ ]:
def CCZ_sequence(initial_id):
    # Define sequence
    seq = Sequence(reg_CCZ, Chadoq2)
    
    # Prepare State
    prep_state = prep_sequence_CCZ(initial_id, seq)
    prep_time = max((seq._last(ch).tf for ch in seq.declared_channels), default=0)
    
    # Declare Rydberg channel
    seq.declare_channel('rydA', 'rydberg_local', 'control1')
    seq.declare_channel('rydB', 'rydberg_local2', 'control2')
    
    # Write CZ sequence:
    seq.add(pi_pulse, 'rydA', 'wait-for-all')  # Wait for state preparation to finish.
    seq.align('rydA','rydB')
    seq.add(pi_pulse, 'rydB')
    seq.target('target','rydA')
    seq.align('rydA','rydB')
    seq.add(twopi_pulse, 'rydA')
    seq.align('rydA','rydB')
    seq.add(pi_pulse, 'rydB')
    seq.target('control1','rydA')
    seq.align('rydA','rydB')
    
    seq.add(pi_pulse,'rydA')

    return seq, prep_state, prep_time

In [ ]:
CCZ_seq, st, t = CCZ_sequence('ggh')
#CCZ_seq.draw()

In [ ]:
for state_id in {''.join(x) for x in product('gh', repeat=3)}:
    # Get CZ sequence
    print(f"\n CCZ gate sequence acting on state {state_id}\n")
    seq, prep_state, prep_time = CCZ_sequence(state_id)
    
    # Construct Simulation instance
    simul = Simulation(seq)
    
    simul.run(progress_bar = True)
    
    data=[st.overlap(prep_state) for st in simul.output.states]
    
    #plt.plot(np.imag(data))
    plt.figure()
    plt.plot(np.real(data))
    plt.xlabel(r"Time [ns]")
    plt.ylabel(fr'$ {state_id} | \psi(t)\rangle$')
    plt.axvspan(0, prep_time, alpha=0.06, color='royalblue')
    plt.title(fr"Action on state $|${state_id}$\rangle$")

In [ ]:
CCZ = {}
for state_id in {''.join(x) for x in product('gh', repeat=3)}:
    seq, prep_st, prep_t = CCZ_sequence(state_id)
    simul = Simulation(seq)
    simul.run()
    final_st = simul.output.states[-1]
    CCZ[state_id] = final_st.overlap(prep_st)

Our results are as expected: only the $|hhh\rangle$ state (which corresponds to a $111$ digital state) gets its phase flipped in sign

In [ ]:
CCZ